# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [53]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [54]:
dataset = pd.read_csv("./data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [43]:
X.shape, y.shape

((1854, 10), (1854,))

In [44]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1483, 10), (371, 10), (1483,), (371,))


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

1. El conjunto de datos trata sobre préstamos y está basado en el conjunto de datos de Kaggle Home Equity. El conjunto de datos trata sobre el valor líquido de la vivienda (HMEQ) contiene información de referencia y de rendimiento de préstamos.
2. La variable objetivo a predecir es TARGET, es una variable binaria que indica si un solicitante eventualmente incurrió en incumplimiento o fue gravemente moroso.
3. Los atributos disponibles para hacer la predicción son:


* LOAN: Monto de la solicitud de préstamo
* MORTDUE: Monto adeudado de la hipoteca existente
* VALUE: Valor de la propiedad actual
* YOJ: Años en el trabajo actual
* DEROG: Número de informes despectivos importantes
* DELINQ: Número de líneas de crédito morosas
* CLAGE: Antigüedad de la línea comercial más antigua en meses
* NINQ: Número de líneas de crédito recientes
* CLNO: Número de líneas de crédito
* DEBTINC: Relación deuda-ingresos
4. Consideramos que los atributos mas determinantes para realizar la predicción son:
* LOAN, MORTDUE, YOJ, DELINQ, DEBTINC

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [55]:
# Selección modelo Descenso de gradiente estocástico
model = SGDClassifier(random_state=123)

# Entrenamiento
model.fit(X_train, y_train)

# Predicciones de X_train
x_train_pred = model.predict(X_train)

# Predicciones de X_test
x_test_pred = model.predict(X_test)

# Error cuadrático sobre los datos de entrenamiento / test
train_error = mean_squared_error(y_train,x_train_pred)
test_error = mean_squared_error(y_test,x_test_pred)

In [56]:
# Exactitud de los datos de entrenamiento comparando con la probabilidad de los datos de entrenamiento
accuracy_score = accuracy_score(y_test,x_test_pred)

# La precision es la proporción de positivos propuestos por el modelo que fueron correctos
precision_score = precision_score(y_test,x_test_pred,zero_division=True)

# La recall es la proporción de positivos correctos que fueron encontrados por el modelo
recall_score = recall_score(y_test,x_test_pred)

# f1 = 
f1_score = f1_score(y_test,x_test_pred)

In [47]:
print('Train error:' , train_error.round(2))
print('Test error:', test_error.round(2))
print('Accuracy score:', accuracy_score.round(2))
print('Precision score:', precision_score.round(2))
print('Recall score:', recall_score.round(2))
print('F1 score:', f1_score.round(2))

Train error: 0.17
Test error: 0.16
Accuracy score: 0.84
Precision score: 1.0
Recall score: 0.0
F1 score: 0.0


In [59]:
# MATRIZ DE CONFUSIÓN
cm = confusion_matrix(y_test, x_test_pred, labels=['LOAN', 'MORTDUE', 'YOJ'])
cm

/home/exequiel/anaconda3/envs/Diplomatura2021/lib/python3.8/site-packages/sklearn/metrics/_classification.py:312: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif np.all([l not in y_true for l in labels]):


ValueError: At least one label specified must be in y_true

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html